# SedonaDB KNN + lonboard

In [2]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [8]:
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=places/type=place/"
)

In [19]:
df.show(3)

┌──────────────┬──────────────────────────┐
│ capital_name ┆         location         │
│     utf8     ┆         geometry         │
╞══════════════╪══════════════════════════╡
│ Austin       ┆ POINT(-97.7431 30.2747)  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Sacramento   ┆ POINT(-121.4944 38.5816) │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Boston       ┆ POINT(-71.0589 42.3601)  │
└──────────────┴──────────────────────────┘


In [10]:
df.to_view("overture_places")

In [11]:
sd.sql("""
SELECT 
  names.primary AS name,
  categories.primary AS category,
  geometry,
  addresses[0]['region']
FROM overture_places
WHERE categories.primary = 'coffee_shop'
LIMIT 5;
""").show()

┌────────────────────────────┬─────────────┬───────────────────────────┬───────────────────────────┐
│            name            ┆   category  ┆          geometry         ┆ overture_places.addresses │
│            utf8            ┆     utf8    ┆          geometry         ┆    [Int64(0)][region]…    │
╞════════════════════════════╪═════════════╪═══════════════════════════╪═══════════════════════════╡
│ Italian Foods and Coffee … ┆ coffee_shop ┆ POINT(79.9649058 7.59914… ┆                           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Atara Restaurant & Coffee… ┆ coffee_shop ┆ POINT(79.7654815 8.24232… ┆                           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Ceylon Coffee Company      ┆ coffee_shop ┆ POINT(80.5830356 7.16505… ┆                           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [24]:
sd.sql("""
SELECT 
  places.names.primary AS name, 
  places.categories.primary AS category, 
  places.geometry 
FROM (
  SELECT ST_SetSRID(ST_GeomFromWKT('POINT(-97.7431 30.2747)'), 4326) AS geom
) AS search_point 
JOIN overture_places AS places 
  ON ST_KNN(search_point.geom, places.geometry, 5, false) 
WHERE places.categories.primary = 'coffee_shop'
""").show()

SedonaError: Internal error: PlaceholderRowExec invalid partition 3 (expected less than 1).
This issue was likely caused by a bug in DataFusion's code. Please help us to resolve this by filing a bug report in our issue tracker: https://github.com/apache/datafusion/issues

In [14]:
df = sd.sql("""
SELECT 
    capital_name, 
    ST_SetSRID(ST_Point(lng, lat), 4326) AS location
FROM (
    VALUES 
        ('Austin', -97.7431, 30.2747),
        ('Sacramento', -121.4944, 38.5816),
        ('Boston', -71.0589, 42.3601)
) AS t(capital_name, lng, lat)
""")

df.show()

┌──────────────┬──────────────────────────┐
│ capital_name ┆         location         │
│     utf8     ┆         geometry         │
╞══════════════╪══════════════════════════╡
│ Austin       ┆ POINT(-97.7431 30.2747)  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Sacramento   ┆ POINT(-121.4944 38.5816) │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Boston       ┆ POINT(-71.0589 42.3601)  │
└──────────────┴──────────────────────────┘


In [15]:
df.to_view("capitals")

In [16]:
sd.sql("""
SELECT 
  c.capital_name,
  p.names.primary AS name,
  p.categories.primary AS category,
  ST_Distance(p.geometry, c.location) AS distance,
  p.geometry
FROM overture_places p
JOIN capitals c
ON p.categories.primary = 'coffee_shop'
  AND ST_KNN(p.geometry, c.location, 5, false)
ORDER BY c.capital_name, distance
""").show()

┌──────────────┬──────────────────────┬─────────────┬───────────────────────┬──────────────────────┐
│ capital_name ┆         name         ┆   category  ┆        distance       ┆       geometry       │
│     utf8     ┆         utf8         ┆     utf8    ┆        float64        ┆       geometry       │
╞══════════════╪══════════════════════╪═════════════╪═══════════════════════╪══════════════════════╡
│ Austin       ┆ Sweetwaters Coffee … ┆ coffee_shop ┆ 0.0008701168484779806 ┆ POINT(-97.7439102 3… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Austin       ┆ Apollo Cafe          ┆ coffee_shop ┆  0.002659806669664969 ┆ POINT(-97.7429184 3… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Austin       ┆ 15th Street Cafe     ┆ coffee_shop ┆ 0.0028700824326153983 ┆ POINT(-97.7428706 3… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [25]:
res = sd.sql("""
WITH ranked_shops AS (
  SELECT 
    c.capital_name,
    p.names.primary AS name,
    p.categories.primary AS category,
    ST_Distance(p.geometry, c.location) AS distance,
    p.geometry,
    ROW_NUMBER() OVER (PARTITION BY c.capital_name ORDER BY ST_Distance(p.geometry, c.location)) AS rank
  FROM overture_places p
  JOIN capitals c
  ON p.categories.primary = 'coffee_shop'
    AND ST_KNN(p.geometry, c.location, 5, false)
)
SELECT capital_name, name, category, distance, geometry
FROM ranked_shops
WHERE rank <= 5
ORDER BY capital_name, rank
""")

In [26]:
res.show()

┌──────────────┬──────────────────────┬─────────────┬───────────────────────┬──────────────────────┐
│ capital_name ┆         name         ┆   category  ┆        distance       ┆       geometry       │
│     utf8     ┆         utf8         ┆     utf8    ┆        float64        ┆       geometry       │
╞══════════════╪══════════════════════╪═════════════╪═══════════════════════╪══════════════════════╡
│ Austin       ┆ Sweetwaters Coffee … ┆ coffee_shop ┆ 0.0008701168484779806 ┆ POINT(-97.7439102 3… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Austin       ┆ Apollo Cafe          ┆ coffee_shop ┆  0.002659806669664969 ┆ POINT(-97.7429184 3… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Austin       ┆ 15th Street Cafe     ┆ coffee_shop ┆ 0.0028700824326153983 ┆ POINT(-97.7428706 3… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [27]:
import lonboard

lonboard.viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…